In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn, tensor
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, average_precision_score
import pandas as pd
from tqdm.notebook import tqdm
import pickle
#import torchsummary
import os
from torch.nn import SmoothL1Loss, MSELoss
from torch.utils.data import TensorDataset, Sampler
#from sklearn.model_selection import train_test_split

pd.set_option('display.max_seq_items', None)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device


In [ ]:
action_num = 16
version = '240721'
date = version
train_data = pd.read_csv(f'./train_shock_dataset_state_cont_{version}_Action{action_num}_Reward_Mean_VM.csv')
val_data = pd.read_csv(f'./val_shock_dataset_state_cont_{version}_Action{action_num}_Reward_Mean_VM.csv')
# param 설정
state_dim = 19 
action_dim = action_num

In [ ]:
columns_list = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18']
for column_name in columns_list:
    train_data = train_data.rename(columns={f'{column_name}': f's:{column_name}'})

for column_name in columns_list:
    val_data = val_data.rename(columns={f'{column_name}': f's:{column_name}'})

In [ ]:
train_data.rename(columns={'Action':'a:Action'}, inplace=True)
train_data.rename(columns={'Reward':'r:Reward'}, inplace=True)
train_data.Shock.fillna(0, inplace=True)
shock_index = train_data[train_data['Target Shock']!=0].index
train_data['sh:Shock'] = 0
train_data['sh:Shock'][shock_index] = 1

def define_terminal(x):
    x.reset_index(drop=True, inplace=True)

    x['t:Terminal'] = 0

    x['t:Terminal'].loc[len(x)-1] = 1

    return x
train_data = train_data.groupby('stay_id').apply(define_terminal)
train_data.reset_index(drop=True, inplace=True)

#data.rename(columns={'State':'s:State'}, inplace=True)
val_data.rename(columns={'Action':'a:Action'}, inplace=True)
val_data.rename(columns={'Reward':'r:Reward'}, inplace=True)
val_data.Shock.fillna(0, inplace=True)
shock_index = val_data[val_data['Target Shock']!=0].index
val_data['sh:Shock'] = 0
val_data['sh:Shock'][shock_index] = 1

val_data = val_data.groupby('stay_id').apply(define_terminal)
val_data.reset_index(drop=True, inplace=True)

In [ ]:
#악화 중
print('Shock 상태 악화 중: ', train_data[(train_data['Blood pressure mean_RSI']<=30) & (train_data['Target Shock']==2)]['Blood pressure mean'].mean())
#약한 회복 중
print('Shock 상태 약한 회복 중: ', train_data[(train_data['Blood pressure mean_RSI']<70) & (train_data['Blood pressure mean_RSI']>30) & (train_data['Target Shock']==2)]['Blood pressure mean'].mean())
#강한 회복 중
print('Shock 상태 강한 회복 중: ', train_data[(train_data['Blood pressure mean_RSI']>=70) & (train_data['Target Shock']==2)]['Blood pressure mean'].mean())

In [ ]:
#악화 중
print('Peri Shock 상태 악화 중: ', train_data[(train_data['Blood pressure mean_RSI']<=30) & (train_data['Target Shock']==1)]['Blood pressure mean'].mean())
#약한 회복 중
print('Peri Shock 상태 약한 회복 중: ', train_data[(train_data['Blood pressure mean_RSI']<70) & (train_data['Blood pressure mean_RSI']>30) & (train_data['Target Shock']==1)]['Blood pressure mean'].mean())
#강한 회복 중
print('Peri Shock 상태 강한 회복 중: ', train_data[(train_data['Blood pressure mean_RSI']>=70) & (train_data['Target Shock']==1)]['Blood pressure mean'].mean())

In [ ]:
#악화 중
print('정상 상태 악화 중: ', train_data[(train_data['Blood pressure mean_RSI']<=30) & (train_data['Target Shock']==0)]['Blood pressure mean'].mean())
#약한 회복 중
print('정상 상태 약한 회복 중: ', train_data[(train_data['Blood pressure mean_RSI']<70) & (train_data['Blood pressure mean_RSI']>30) & (train_data['Target Shock']==0)]['Blood pressure mean'].mean())
#강한 회복 중
print('정상 상태 강한 회복 중: ', train_data[(train_data['Blood pressure mean_RSI']>=70) & (train_data['Target Shock']==0)]['Blood pressure mean'].mean())

Reward definition

In [ ]:
'''#Dead/Alive in terminal mode
#control reward train
train_data['r:Reward'] = 0
train_data['r:Reward'][train_data[(train_data['t:Terminal']==1)].index] = 100
train_data['r:Reward'][train_data[(train_data['Death']==1) & (train_data['t:Terminal']==1)].index] = -100
print('Train: ', train_data['r:Reward'].unique())

#control reward val
val_data['r:Reward'] = 0
val_data['r:Reward'][val_data[(val_data['t:Terminal']==1)].index] = 100
val_data['r:Reward'][val_data[(val_data['Death']==1) & (val_data['t:Terminal']==1)].index] = -100
print('Validation: ',val_data['r:Reward'].unique())'''

In [ ]:
#Reward:Labtest
train_data['Reward:vl'] = 0
#Lactate
train_data['Reward:Lactate'] = 0
train_data['Reward:Lactate'][train_data[(train_data['Lactate']>2)&(train_data['Lactate Elevation index']==2)].index] = -10
train_data['Reward:Lactate'][train_data[(train_data['Lactate']<=2)&(train_data['Lactate Elevation index']==-2)].index] = 10
#Creatinine
train_data['Reward:Creatinine'] = 0
train_data['Reward:Creatinine'][train_data[(train_data['Creatinine']>3)&(train_data['Creatinine Elevation index']==2)].index] = -10
train_data['Reward:Creatinine'][train_data[(train_data['Creatinine']<=3)&(train_data['Creatinine Elevation index']==-2)].index] = 10
#Troponin
train_data['Reward:Troponin'] = 0
train_data['Reward:Troponin'][train_data[(train_data['Troponin']>14)&(train_data['Troponin Elevation index']==2)].index] = -10
train_data['Reward:Troponin'][train_data[(train_data['Troponin']<=14)&(train_data['Troponin Elevation index']==-2)].index] = 10

train_data['Reward:vl'] = train_data['Reward:Lactate'] + train_data['Reward:Creatinine'] + train_data['Reward:Troponin']

#MAP mode
train_data['Reward:MAP'] = 0
#악화 중(shock)
train_data['Reward:MAP'][train_data[(train_data['Blood pressure mean_RSI']<30) & (train_data['Target Shock']==2)].index] = -10
#약한 회복 중(shock)
#train_data['Reward:MAP'][train_data[(train_data['Blood pressure mean_RSI']<70) & (train_data['Blood pressure mean_RSI']>30) & (train_data['Target Shock']==2)].index] = -50
#강한 회복 중(shock)
train_data['Reward:MAP'][train_data[(train_data['Blood pressure mean_RSI']>70) & (train_data['Target Shock']==2)].index] = 10

#악화 중(peri-shock)
train_data['Reward:MAP'][train_data[(train_data['Blood pressure mean_RSI']<30) & (train_data['Target Shock']==1)].index] =-10
#약한 회복 중(peri-shock)
#train_data['Reward:MAP'][train_data[(train_data['Blood pressure mean_RSI']<70) & (train_data['Blood pressure mean_RSI']>30) & (train_data['Target Shock']==1)].index] = -50
#강한 회복 중(peri-shock)
train_data['Reward:MAP'][train_data[(train_data['Blood pressure mean_RSI']>70) & (train_data['Target Shock']==1)].index] = 10

#Dead/Alive in terminal mode
#control reward train
train_data['Reward:Terminal'] = 0
train_data['Reward:Terminal'][train_data[(train_data['t:Terminal']==1)].index] = 100
train_data['Reward:Terminal'][train_data[(train_data['Death']==1) & (train_data['t:Terminal']==1)].index] = -100
print('Train: ', train_data['Reward:Terminal'].unique())

train_data['r:Reward'] = 0
train_data['r:Reward'] = train_data['Reward:MAP'] + train_data['Reward:vl'] + train_data['Reward:Terminal']
print('Train: ', train_data['r:Reward'].unique())

In [ ]:
#Reward:Labtest
val_data['Reward:vl'] = 0
#Lactate
val_data['Reward:Lactate'] = 0
val_data['Reward:Lactate'][val_data[(val_data['Lactate']>2)&(val_data['Lactate Elevation index']==2)].index] = -10
val_data['Reward:Lactate'][val_data[(val_data['Lactate']<=2)&(val_data['Lactate Elevation index']==-2)].index] = 10
#Creatinine
val_data['Reward:Creatinine'] = 0
val_data['Reward:Creatinine'][val_data[(val_data['Creatinine']>3)&(val_data['Creatinine Elevation index']==2)].index] = -10
val_data['Reward:Creatinine'][val_data[(val_data['Creatinine']<=3)&(val_data['Creatinine Elevation index']==-2)].index] = 10
#Troponin
val_data['Reward:Troponin'] = 0
val_data['Reward:Troponin'][val_data[(val_data['Troponin']>14)&(val_data['Troponin Elevation index']==2)].index] = -10
val_data['Reward:Troponin'][val_data[(val_data['Troponin']<=14)&(val_data['Troponin Elevation index']==-2)].index] = 10

val_data['Reward:vl'] = val_data['Reward:Lactate'] + val_data['Reward:Creatinine'] + val_data['Reward:Troponin']

#MAP mode
val_data['Reward:MAP'] = 0
#악화 중(shock)
val_data['Reward:MAP'][val_data[(val_data['Blood pressure mean_RSI']<30) & (val_data['Target Shock']==2)].index] = -10
#약한 회복 중(shock)
#val_data['Reward:MAP'][val_data[(val_data['Blood pressure mean_RSI']<70) & (val_data['Blood pressure mean_RSI']>30) & (val_data['Target Shock']==2)].index] = -50
#강한 회복 중(shock)
val_data['Reward:MAP'][val_data[(val_data['Blood pressure mean_RSI']>70) & (val_data['Target Shock']==2)].index] = 10

#악화 중(peri-shock)
val_data['Reward:MAP'][val_data[(val_data['Blood pressure mean_RSI']<30) & (val_data['Target Shock']==1)].index] =-10
#약한 회복 중(peri-shock)
#val_data['Reward:MAP'][val_data[(val_data['Blood pressure mean_RSI']<70) & (val_data['Blood pressure mean_RSI']>30) & (val_data['Target Shock']==1)].index] = -50
#강한 회복 중(peri-shock)
val_data['Reward:MAP'][val_data[(val_data['Blood pressure mean_RSI']>70) & (val_data['Target Shock']==1)].index] = 10

#control reward val
val_data['Reward:Terminal'] = 0
val_data['Reward:Terminal'][val_data[(val_data['t:Terminal']==1)].index] = 100
val_data['Reward:Terminal'][val_data[(val_data['Death']==1) & (val_data['t:Terminal']==1)].index] = -100
print('Validation: ',val_data['Reward:Terminal'].unique())

val_data['r:Reward'] = 0
val_data['r:Reward'] = val_data['Reward:MAP'] + val_data['Reward:vl'] + val_data['Reward:Terminal']
print('Validation: ', val_data['r:Reward'].unique())

In [ ]:
print('Train reward ratio: ', train_data['r:Reward'].value_counts()/len(train_data))
print('Validation reward ratio: ', val_data['r:Reward'].value_counts()/len(val_data))

In [ ]:
print('Train reward min: ', train_data['r:Reward'].min())
print('Train reward max: ', train_data['r:Reward'].max())
print('Train reward mean: ', train_data['r:Reward'].mean())

print('Validation reward min: ', val_data['r:Reward'].min())
print('Validation reward max: ', val_data['r:Reward'].max())
print('Validation reward mean: ', val_data['r:Reward'].mean())

In [ ]:
def load_dataset(df):
    # state, action, reward, next_state로 데이터를 나누기
    states = df['s:State'].values.reshape(-1, 1)  # state는 1차원으로 reshape
    actions = df['a:Action'].values
    rewards = df['r:Reward'].values
    next_states = df['s_prime:Next_state'].values.reshape(-1, 1)  # next_state는 1차원으로 reshape
    terminal = df['t:Terminal'].values
    
    return states, actions, rewards, next_states, terminal

def one_hot_encode(data, num_classes):
    """
    Perform one-hot encoding on the given data.
    :param data: Data to be encoded.
    :param num_classes: Number of classes for one-hot encoding.
    :return: Encoded data.
    """
    encoded_data = np.zeros((len(data), num_classes))
    for i, val in enumerate(data):
        val = int(val)  # Ensure val is an integer
        encoded_data[i, val] = 1
    return encoded_data

def one_hot_encode_for_val(data, num_classes):
    """
    Perform one-hot encoding on the given data.
    :param data: Data to be encoded.
    :param num_classes: Number of classes for one-hot encoding.
    :return: Encoded data.
    """
    encoded_data = np.zeros((len(data), num_classes))
    for i, val in enumerate(data):
        #val = int(val)  # Ensure val is an integer
        encoded_data[i, val] = 1
    return encoded_data

def make_transition(data):
    df = data
    s_col = [x for x in df if x[:2] == 's:']
    a_col = [x for x in df if x[:2] == 'a:']
    r_col = [x for x in df if x[:2] == 'r:']
    t_col = [x for x in df if x[:2] == 't:']
    
    dict = {}
    dict['traj'] = {}
    data_len = 0
    s, a, r, s2, t  = [], [], [], [], [] #

    for traj in tqdm(df.stay_id.unique()):
        df_traj = df[df['stay_id'] == traj]
        dict['traj'][traj] = {'s': [], 'a': [], 'r': [], 't': []}
        dict['traj'][traj]['s'] = df_traj[s_col].values.tolist()
        dict['traj'][traj]['a'] = df_traj[a_col].values.tolist()
        dict['traj'][traj]['r'] = df_traj[r_col[0]].values.tolist()
        dict['traj'][traj]['t'] = df_traj[t_col].values.tolist()
        #print(dict['traj'][traj]['s'][step:step+rolling_size])
        step_len = len(df_traj) - 1
        for step in range(step_len):
            s.append(dict['traj'][traj]['s'][step])
            a.append(dict['traj'][traj]['a'][step])
            r.append(dict['traj'][traj]['r'][step+1])
            s2.append(dict['traj'][traj]['s'][step+1])
            t.append(dict['traj'][traj]['t'][step+1])
            data_len += 1

    s = torch.FloatTensor(np.float32(s))
    a = torch.LongTensor(np.int64(a))
    r = torch.FloatTensor(np.float32(r))
    s2 = torch.FloatTensor(np.float32(s2))
    t = torch.FloatTensor(np.float32(t))

    return s,a,r,s2,t

class CustomSampler(Sampler):
    def __init__(self, data, batch_size, ns, target):
        self.data = data
        self.batch_size = batch_size
        self.num_samples_1 = ns
        self.num_samples_0 = batch_size - ns
        self.indices = [i for i in range(len(data)) if data[i][2].item() in target]
        self.indices_neg = [i for i in range(len(data)) if data[i][2].item() not in target]
        self.used_indices_neg = []

    def __iter__(self):
        np.random.shuffle(self.indices)
        np.random.shuffle(self.indices_neg)

        batch = []
        for idx in self.indices:
            batch.append(idx)
            if len(batch) == self.num_samples_0:
                batch.extend(self._sample_indices_neg())
                yield batch
                batch = []

    def _sample_indices_neg(self, remaining=0):
        if remaining:
            num_samples_1 = self.batch_size - remaining
        else:
            num_samples_1 = self.num_samples_1

        if len(self.used_indices_neg) + num_samples_1 > len(self.indices_neg):
            self.used_indices_neg = []
            np.random.shuffle(self.indices_neg)

        indices_neg = self.indices_neg[len(self.used_indices_neg):len(self.used_indices_neg) + num_samples_1]
        self.used_indices_neg.extend(indices_neg)
        return indices_neg

    def __len__(self):
        return (len(self.indices) + len(self.indices_neg)) // self.batch_size

In [ ]:
def param():
    USER_NAME = 'JH'                  
    SEED      = 9861
    batch_size= 128
    epoch     = 200
    lr        = 0.0001
    drop_rate = 0
    hidden = {
        'hidden' : []
    }
    return hidden, SEED, batch_size, lr, epoch, drop_rate

def init_weights(m):
    if type(m) in [nn.Linear, nn.Conv2d]:
        torch.nn.init.kaiming_uniform_(m.weight)
        m.bias.data.fill_(0.)

class C51DQNDense(nn.Module):
    def __init__(self, state_dim, action_dim, N_ATOM, hidden_sizes):
        super(C51DQNDense, self).__init__()
        self.action_dim = action_dim
        self.N_ATOM = N_ATOM

        self.layer = nn.ModuleList([
            nn.Linear(in_features=state_dim, out_features=hidden_sizes[0]),
            nn.BatchNorm1d(hidden_sizes[0]),
            nn.ELU()
        ])
        
        for i in range(1, len(hidden_sizes)):
            self.layer.extend([
                nn.Linear(in_features=hidden_sizes[i-1], out_features=hidden_sizes[i]),
                nn.BatchNorm1d(hidden_sizes[i]),
                nn.ELU()
            ])

        self.layer.append(nn.Linear(in_features=hidden_sizes[-1], out_features=action_dim * N_ATOM))

    def forward(self, x):
        mb_size = x.size(0)

        for layer in self.layer:
            x = layer(x)
    
        action_value = F.softmax(x.view(mb_size, action_dim, N_ATOM), dim=2)
        
        return action_value
    
    def save(self, PATH):
        torch.save(self,PATH)

    def load(self, PATH):
        self.load_state_dict(torch.load(PATH))

def objective(trial):
    # Define the hyperparameters to be optimized
    hidden_sizes = {
        'hidden': []
    }
    for idx in range(trial.suggest_int('num_layers', 2,5)):
        hidden_sizes['hidden'].append(trial.suggest_int(f'h_{idx}', 10, 200))
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    lr = trial.suggest_categorical('lr', [0.000001, 0.00001, 0.00003, 0.00005, 0.0001])  # ,0.00001,0.00003,0.00005 ,0.0001,0.0003,
    epoch_num = trial.suggest_categorical('epoch', [50])
    gamma = trial.suggest_categorical('gamma', [0.99, 0.9, 0.8])
    #drop_rate = trial.suggest_uniform('drop_rate', 0.0, 0.1)
    update_interval = trial.suggest_int('update_interval', 2, 20)
    #update_interval = trial.suggest_int('update_interval', 200, 300)

    # Initialize your network with the suggested hyperparameters
    print('============================================================')
    print('state_dim: ', state_dim)
    print('action_dim: ', action_dim)
    print('learning_rate: ',lr)
    print('hidden_sizes: ', hidden_sizes)
    print('gamma: ', gamma)
    #print('drop_rate: ', drop_rate)
    print('batch_size: ', batch_size)
    print('epoch_num: ', epoch_num)
    print('update_interval: ', update_interval)
    
    network = C51OffDQNAgent(state_dim=state_dim, action_dim=action_dim, learning_rate=lr, update_interval=update_interval, hidden_sizes=hidden_sizes['hidden'], gamma=gamma) #, drop_out=drop_rate
    print('------------------------------------------------------------')
    print(network.policy)
    print('============================================================')
    # Train the network and get validation loss
    val_loss = train_and_evaluate(network=network, train_transition=train_transition, val_transition=val_transition, batch_size=batch_size,  epoch_num=epoch_num)
    
    return val_loss

V_MIN = -140.
V_MAX = 140.
N_ATOM = 51
V_RANGE = np.linspace(V_MIN, V_MAX, N_ATOM)
V_STEP = ((V_MAX-V_MIN)/(N_ATOM-1))

PRED_PATH = f'./Model VM/vm_C51_pred_net_{date}.pt'
TARGET_PATH = f'./Model VM/vm_C51_target_net_{date}.pt'
RESULT_PATH = f'./Model VM/vm_C51_result_{date}.pt'

class C51OffDQNAgent(object):
    def __init__(self, state_dim, action_dim, hidden_sizes, update_interval, gamma, N_ATOM=51, learning_rate=0.0005):
        #super().__init__(state_dim, action_dim)
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.name = 'C51OffDQNAgent'
        self.batches_done = 0
        self.learning_rate = learning_rate

        self.target_update_steps = update_interval
        self.gamma = gamma
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

        self.N_ATOM = N_ATOM

        print('Utilizing device {}'.format(self.device))
        self.policy = C51DQNDense(state_dim=state_dim, action_dim=action_dim, N_ATOM=N_ATOM, hidden_sizes=hidden_sizes).to(self.device)
        self.target = C51DQNDense(state_dim=state_dim, action_dim=action_dim, N_ATOM=N_ATOM, hidden_sizes=hidden_sizes).to(self.device)
        self.target.load_state_dict(self.policy.state_dict())
        self.target.eval()
        self.optimizer = torch.optim.Adam(self.policy.parameters(), lr=learning_rate)

        # discrete values
        self.value_range = torch.FloatTensor(V_RANGE).to(self.device)

    def save_model(self):
        # save prediction network and target network
        self.policy.save(PRED_PATH)
        self.target.save(TARGET_PATH)

    def load_model(self):
        # load prediction network and target network
        self.policy.load(PRED_PATH)
        self.target.load(TARGET_PATH)

    def act(self, state: np.ndarray) -> np.ndarray:
        with torch.no_grad():
            self.policy.eval()
            state = torch.tensor(state).float().unsqueeze(0).to(self.device)
            action = torch.argmax(self.policy(state))
            return action.cpu().detach().numpy()

    def train_batch(self, states, actions, rewards, next_states, terminals):
        self.policy.train()

        state = torch.tensor(states, dtype=torch.float32).to(self.device)
        action = torch.tensor(actions, dtype=torch.long).to(self.device)
        reward = torch.tensor(rewards, dtype=torch.float32).to(self.device)
        new_state = torch.tensor(next_states, dtype=torch.float32).to(self.device)
        done = torch.tensor(terminals, dtype=torch.long).to(self.device)

        weight, b_idxes = np.ones_like(rewards), None

        # action value distribution prediction
        q_eval = self.policy(state) # (m, N_ACTIONS, N_ATOM) # (m, N_ACTIONS, N_ATOM) 
        mb_size = q_eval.size(0)
        q_eval = torch.stack([q_eval[i].index_select(0, action[i]) for i in range(mb_size)]).squeeze(1) # (m, N_ATOM)
        
        # target distribution
        q_target = np.zeros((mb_size, N_ATOM)) # (m, N_ATOM)
        
        # get next state value
        q_next = self.target(new_state).detach() # (m, N_ACTIONS, N_ATOM)
        # next value mean
        q_next_mean = torch.sum(q_next * self.value_range.view(1, 1, -1), dim=2) # (m, N_ACTIONS)

        
        best_actions = q_next_mean.argmax(dim=1) # (m)
        q_next = torch.stack([q_next[i].index_select(0, best_actions[i]) for i in range(mb_size)]).squeeze(1) 
        q_next = q_next.data.cpu().numpy() # (m, N_ATOM)
        

        # categorical projection
        '''
        next_v_range : (z_j) i.e. values of possible return, shape : (m, N_ATOM)
        next_v_pos : relative position when offset of value is V_MIN, shape : (m, N_ATOM)
        '''
        # we vectorized the computation of support and position
        reward_np = reward.data.cpu().numpy()
        done_np = done.data.cpu().numpy()
        value_range_np = self.value_range.data.cpu().numpy()

        reward_np = np.expand_dims(reward_np, 1)
        done_np = np.expand_dims((1. - done_np), 1)
        #value_range_np = np.expand_dims(value_range_np, 0)

        next_v_range = reward_np + (self.gamma * done_np * value_range_np).squeeze(1)

        #next_v_range = np.expand_dims(reward.data.cpu().numpy(), 1) + gamma * np.expand_dims((1. - done.data.cpu().numpy()),1) * np.expand_dims(self.value_range.data.cpu().numpy(),0)
        
        next_v_pos = np.zeros_like(next_v_range)
        #print('next_v_pos:', next_v_pos.shape)

        # clip for categorical distribution
        next_v_range = np.clip(next_v_range, V_MIN, V_MAX)
        
        # calc relative position of possible value
        next_v_pos = (next_v_range - V_MIN)/ V_STEP
        # get lower/upper bound of relative position
        lb = np.floor(next_v_pos).astype(int)
        ub = np.ceil(next_v_pos).astype(int)
        # we didn't vectorize the computation of target assignment.
        #print('lb: ', lb.shape)
        #print('ub: ', ub.shape)
        #print('mb_size: ', mb_size)
        #print('N_ATOM: ', N_ATOM)
        #print('q_next shape: ', q_next.shape)
        for i in range(mb_size):
            for j in range(N_ATOM):
                # calc prob mass of relative position weighted with distance
                q_target[i, lb[i,j]] += (q_next * (ub - next_v_pos))[i,j]
                q_target[i, ub[i,j]] += (q_next * (next_v_pos - lb))[i,j]
                
        q_target = torch.FloatTensor(q_target).to(self.device) 
        
        loss = q_target * ( - torch.log(q_eval + 1e-10)) # (m , N_ATOM)
        loss = torch.mean(loss)

        '''weight = torch.Tensor(weight).to(self.device) 
        loss = torch.mean(weight*loss)'''
        
        
        # backprop loss
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()


        # Update target every n steps
        if self.batches_done % self.target_update_steps == 0:
            self.target.load_state_dict(self.policy.state_dict())

        self.batches_done += 1

        return loss.item()


    def validate_batch_loss(self, states, actions, rewards, next_states, terminals):
        with torch.no_grad():
            self.policy.eval()

            state = torch.tensor(states, dtype=torch.float32).to(self.device)
            action = torch.tensor(actions, dtype=torch.long).to(self.device)
            reward = torch.tensor(rewards, dtype=torch.float32).to(self.device)
            new_state = torch.tensor(next_states, dtype=torch.float32).to(self.device)
            done = torch.tensor(terminals, dtype=torch.long).to(self.device)

            weight, b_idxes = np.ones_like(rewards), None

            # action value distribution prediction
            q_eval = self.policy(state) # (m, N_ACTIONS, N_ATOM)
            mb_size = q_eval.size(0)
            q_eval = torch.stack([q_eval[i].index_select(0, action[i]) for i in range(mb_size)]).squeeze(1) # (m, N_ATOM)
            
            # target distribution
            q_target = np.zeros((mb_size, N_ATOM)) # (m, N_ATOM)
            
            # get next state value
            q_next = self.target(new_state).detach() # (m, N_ACTIONS, N_ATOM)
            # next value mean
            q_next_mean = torch.sum(q_next * self.value_range.view(1, 1, -1), dim=2) # (m, N_ACTIONS)
            best_actions = q_next_mean.argmax(dim=1) # (m)
            q_next = torch.stack([q_next[i].index_select(0, best_actions[i]) for i in range(mb_size)]).squeeze(1) 
            q_next = q_next.data.cpu().numpy() # (m, N_ATOM)

            # categorical projection
            '''
            next_v_range : (z_j) i.e. values of possible return, shape : (m, N_ATOM)
            next_v_pos : relative position when offset of value is V_MIN, shape : (m, N_ATOM)
            '''
            # we vectorized the computation of support and position
            reward_np = reward.data.cpu().numpy()
            done_np = done.data.cpu().numpy()
            value_range_np = self.value_range.data.cpu().numpy()

            reward_np = np.expand_dims(reward_np, 1)
            done_np = np.expand_dims((1. - done_np), 1)
            #value_range_np = np.expand_dims(value_range_np, 0)

            next_v_range = reward_np + (self.gamma * done_np * value_range_np).squeeze(1)
            #next_v_range = np.expand_dims(reward.data.cpu().numpy(), 1) + gamma * np.expand_dims((1. - done.data.cpu().numpy()),1) * np.expand_dims(self.value_range.data.cpu().numpy(),0)
            next_v_pos = np.zeros_like(next_v_range)
            
            
            # clip for categorical distribution
            next_v_range = np.clip(next_v_range, V_MIN, V_MAX)
            # calc relative position of possible value
            next_v_pos = (next_v_range - V_MIN)/ V_STEP
            # get lower/upper bound of relative position
            lb = np.floor(next_v_pos).astype(int)
            ub = np.ceil(next_v_pos).astype(int)
            # we didn't vectorize the computation of target assignment.
            for i in range(mb_size):
                for j in range(N_ATOM):
                    # calc prob mass of relative position weighted with distance
                    q_target[i, lb[i,j]] += (q_next * (ub - next_v_pos))[i,j]
                    q_target[i, ub[i,j]] += (q_next * (next_v_pos - lb))[i,j]
                    
            q_target = torch.FloatTensor(q_target).to(self.device) 

            loss = q_target * ( - torch.log(q_eval + 1e-8)) # (m , N_ATOM)
            loss = torch.mean(loss)

            '''weight = torch.Tensor(weight).to(self.device) 
            loss = torch.mean(weight*loss)'''

            return loss.item()

    def validate_batch_auroc(self, states, actions, label):
        with torch.no_grad():
            self.policy.eval()

            state = torch.tensor(states, dtype=torch.float32).to(self.device)
            action = torch.tensor(actions, dtype=torch.long).to(self.device)
            label = torch.tensor(label, dtype=torch.float32).to(self.device)

            q_value = self.policy(state).detach() # (m, N_ACTIONS, N_ATOM)
            q_value_mean = torch.sum(q_value * self.value_range.view(1, 1, -1), dim=2) # (m, N_ACTIONS)
            q_max = torch.max((q_value_mean+100)/200, dim=1)  
            q_max_ls = [[val.item()] for val in q_max[0]]
            
            q_max_array = np.array(q_max_ls)
            
            label_array = label.data.cpu().numpy()
            
            auroc = roc_auc_score(label_array, -q_max_array) #
            auprc = average_precision_score(label_array, -q_max_array)

            return auroc, auprc
    
    def plot_q_value_distribution(self, state):
        with torch.no_grad():
            self.policy.eval()
            state_tensor = torch.tensor(state, dtype=torch.float32).to(self.device)
            q_dist = self.policy(state_tensor).detach().data.cpu().numpy() # (m, N_ACTIONS, N_ATOM)
            print('q_dist: ', q_dist.shape)

            q_value = self.policy(state_tensor).detach() # (m, N_ACTIONS, N_ATOM)
            q_value_mean = torch.sum(q_value * self.value_range.view(1, 1, -1), dim=2) # (m, N_ACTIONS)
            #q_dist = q_dist.squeeze().detach().numpy()  # 배치 차원 제거하고 numpy 배열로 변환
        
        #actions = list(range(q_dist.shape[1]))
        atoms_ls = np.linspace(V_MIN, V_MAX, 51)
        for batch_num in range(q_dist.shape[0]):
            for action_num in range(q_dist.shape[1]):
                q_probabilities = q_dist[batch_num,action_num]
                q_value_mean_in_batch = q_value_mean[batch_num]
                plt.bar(atoms_ls, q_probabilities)
                plt.xlabel('Atoms')
                plt.ylabel('Prob')
                plt.title(f'Q-value Distribution for State-Action {action_num}, Q value is {q_value_mean_in_batch[action_num]}')
                plt.savefig(f'./Q_distribution graph/VM/Q-value Distribution for State-Action {action_num}')
                plt.show()
                print('------------------------------------------------')
                print(f'Action {action_num} Q value: ', q_value_mean_in_batch[action_num])
                print('================================================')

    def print_q_value_distribution(self, state):
        with torch.no_grad():
            self.policy.eval()
            state_tensor = torch.tensor(state, dtype=torch.float32).to(self.device)
            q_dist = self.policy(state_tensor).detach().data.cpu().numpy() # (m, N_ACTIONS, N_ATOM)
            
            q_value = self.policy(state_tensor).detach() # (m, N_ACTIONS, N_ATOM)
            q_value_mean = torch.sum(q_value * self.value_range.view(1, 1, -1), dim=2) # (m, N_ACTIONS)

            return q_dist, q_value_mean


    def print_policy(self) :
        dict = {}
        with torch.no_grad():
            self.policy.eval()
            for state in range(self.state_dim):
                state_ls = [state]
                state_one_hot = one_hot_encode(data=state_ls, num_classes=state_dim)
                state_tensor = torch.tensor(state_one_hot, dtype=torch.float32)
                q_value = network.policy(state_tensor)
                q_value_mean = torch.sum(q_value * self.value_range.view(1, 1, -1), dim=2) # (m, N_ACTIONS)
                best_actions = q_value_mean.argmax(dim=1)
                action = best_actions.item()
                dict[state] = action

def train_and_evaluate(network, train_transition, val_transition, batch_size, epoch_num):
    torch.manual_seed(220604)
    np.random.seed(220604)
    '''train_data_loader = DataLoader(dataset=train_transition,
                                batch_size=batch_size,
                                shuffle=True,
                                num_workers=4,
                                pin_memory=True)'''
    sampler = CustomSampler(data=train_transition, batch_size=batch_size, ns=int(batch_size*0.05), target=[0.0, 10.0, 20.0, 30.0])
    train_data_loader = DataLoader(train_transition, batch_sampler=sampler, num_workers=4)
        
    val_data_loader = DataLoader(dataset=val_transition,
                            batch_size=batch_size,
                            shuffle=False,
                            num_workers=4,
                            pin_memory=True)
    # DataLoader 생성
    total_loss = []
    Total_num = 0
    

    epoch_loss = []
    epoch_val_loss = []
    epoch_val_auroc = []
    epoch_val_auprc = []
    Total_num += 1
    for epoch in tqdm(range(1, epoch_num+1)):  # 에폭 수 설정
        batch_num = 0
        batch_loss = []
        #train
        for i_batch, batch_data in enumerate(train_data_loader):
            batch_num += 1
            train_states, train_actions, train_rewards, train_next_states, train_terminal = batch_data
            #print('train_states', train_states.shape)
            loss = network.train_batch(train_states, train_actions, train_rewards, train_next_states, train_terminal)
            #print(f'Batch {batch_num} Loss : {loss}')
            #print('------------------------------------------------------------')
            batch_loss.append(loss)

        reward = []
        q_value = []

        val_batch_num = 0
        val_batch_loss = []
        val_batch_auroc = []
        val_batch_auprc = []
        for val_i_batch, val_batch_data in enumerate(val_data_loader):
            val_batch_num += 1
            val_states, val_actions, val_rewards, val_next_states, val_terminal = val_batch_data
            #reward.append(val_rewards)
            #network.policy(val_states), q_max
            #q_value = []
            #AUROC
            val_loss = network.validate_batch_loss(val_states, val_actions, val_rewards, val_next_states, val_terminal) #
            #print(f'Batch {batch_num} Loss : {loss}')
            #print('------------------------------------------------------------')
            val_batch_loss.append(val_loss)
            #val_batch_auroc.append(val_auroc)
        
        for val_i_batch_auroc, val_batch_data_auroc in enumerate(total_val_transition):
            val_batch_num += 1
            val_states, val_actions, val_labels = val_batch_data_auroc
            #AUROC
            val_auroc, val_auprc = network.validate_batch_auroc(val_states, val_actions, val_labels) #
            val_batch_auroc.append(val_auroc)
            val_batch_auprc.append(val_auprc)

        batch_loss_mean = np.mean(batch_loss)
        batch_val_loss_mean = np.mean(val_batch_loss)
        batch_val_auroc_mean = np.mean(val_batch_auroc)
        batch_val_auprc_mean = np.mean(val_batch_auprc)
        print(f'Epoch {epoch} loss : {batch_loss_mean}')
        print(f'Epoch {epoch} val_loss : {batch_val_loss_mean}')
        print(f'Epoch {epoch} val_auroc : {batch_val_auroc_mean}')
        print(f'Epoch {epoch} val_auprc : {batch_val_auprc_mean}')
        print('------------------------------------------------------------')
        epoch_loss.append(batch_loss_mean)
        epoch_val_loss.append(batch_val_loss_mean)
        epoch_val_auroc.append(batch_val_auroc_mean)
        epoch_val_auprc.append(batch_val_auprc_mean)
    
    y1 = epoch_val_loss
    y2 = epoch_val_auroc
    y3 = epoch_val_auprc
    x = list(range(1,len(y1)+1))
    
    plt.figure(figsize=(15, 6))

    # 첫 번째 서브플롯
    plt.subplot(1, 3, 1)
    plt.plot(x, y1, marker='x', label='Epoch Val loss', color='limegreen', linestyle='-')
    plt.title("Epoch Val loss")
    plt.xlabel("epoch")
    plt.ylabel("Val_loss")
    plt.legend()

    # 두 번째 서브플롯
    plt.subplot(1, 3, 2)
    plt.plot(x, y2, marker='o', label='Epoch Val AUROC', color='blue', linestyle='-')
    plt.title("Epoch Val AUROC")
    plt.xlabel("epoch")
    plt.ylabel("Val_auroc")
    plt.legend()

    # tp 번째 서브플롯
    plt.subplot(1, 3, 3)
    plt.plot(x, y3, marker='o', label='Epoch Val AUPRC', color='blue', linestyle='-')
    plt.title("Epoch Val AUPRC")
    plt.xlabel("epoch")
    plt.ylabel("Val_auprc")
    plt.legend()

    plt.show()

    #validataion loss 추가
    epoch_loss_mean = np.mean(epoch_loss)
    epoch_val_loss_mean = np.mean(epoch_val_loss)
    epoch_val_auroc_mean = np.mean(epoch_val_auroc)
    epoch_val_auprc_mean = np.mean(epoch_val_auprc)
    print('Total val_loss_mean: ', epoch_val_loss_mean)
    print('Total val_auroc_mean: ', epoch_val_auroc_mean)
    print('Total val_auprc_mean: ', epoch_val_auprc_mean)
    #print(f'Total {Total_num} loss : {epoch_loss_mean}')
    #print(f'Total {Total_num} val_loss : {epoch_val_loss_mean}')
    #print('============================================================')

    return epoch_val_auroc_mean

def train_and_evaluate_model(network, train_transition, val_transition, batch_size, epoch_num):
    torch.manual_seed(220604)
    np.random.seed(220604)
    '''train_data_loader = DataLoader(dataset=train_transition,
                                batch_size=batch_size,
                                shuffle=True,
                                num_workers=4,
                                pin_memory=True)'''
    sampler = CustomSampler(data=train_transition, batch_size=batch_size, ns=int(batch_size*0.05), target=[0.0, 10.0, 20.0, 30.0])
    train_data_loader = DataLoader(train_transition, batch_sampler=sampler, num_workers=4)
        
    val_data_loader = DataLoader(dataset=val_transition,
                            batch_size=batch_size,
                            shuffle=False,
                            num_workers=4,
                            pin_memory=True)
    # DataLoader 생성
    total_loss = []
    Total_num = 0
    

    epoch_loss = []
    epoch_val_loss = []
    epoch_val_auroc = []
    epoch_val_auprc = []
    Total_num += 1
    patience_value = 0
    patience = 5
    for epoch in tqdm(range(1, epoch_num+1)):  # 에폭 수 설정
        batch_num = 0
        batch_loss = []
        #train
        for i_batch, batch_data in enumerate(train_data_loader):
            batch_num += 1
            train_states, train_actions, train_rewards, train_next_states, train_terminal = batch_data
            #print('train_states', train_states.shape)
            loss = network.train_batch(train_states, train_actions, train_rewards, train_next_states, train_terminal)
            #print(f'Batch {batch_num} Loss : {loss}')
            #print('------------------------------------------------------------')
            batch_loss.append(loss)

        reward = []
        q_value = []

        val_batch_num = 0
        val_batch_loss = []
        val_batch_auroc = []
        val_batch_auprc = []
        for val_i_batch, val_batch_data in enumerate(val_data_loader):
            val_batch_num += 1
            val_states, val_actions, val_rewards, val_next_states, val_terminal = val_batch_data
            #reward.append(val_rewards)
            #network.policy(val_states), q_max
            #q_value = []
            #AUROC
            val_loss = network.validate_batch_loss(val_states, val_actions, val_rewards, val_next_states, val_terminal) #
            #print(f'Batch {batch_num} Loss : {loss}')
            #print('------------------------------------------------------------')
            val_batch_loss.append(val_loss)
            #val_batch_auroc.append(val_auroc)
        
        for val_i_batch_auroc, val_batch_data_auroc in enumerate(total_val_transition):
            val_batch_num += 1
            val_states, val_actions, val_labels = val_batch_data_auroc
            #AUROC
            val_auroc, val_auprc = network.validate_batch_auroc(val_states, val_actions, val_labels) #
            val_batch_auroc.append(val_auroc)
            val_batch_auprc.append(val_auprc)

        batch_loss_mean = np.mean(batch_loss)
        batch_val_loss_mean = np.mean(val_batch_loss)
        batch_val_auroc_mean = np.mean(val_batch_auroc)
        batch_val_auprc_mean = np.mean(val_batch_auprc)
        print(f'Epoch {epoch} loss : {batch_loss_mean}')
        print(f'Epoch {epoch} val_loss : {batch_val_loss_mean}')
        print(f'Epoch {epoch} val_auroc : {batch_val_auroc_mean}')
        print(f'Epoch {epoch} val_auprc : {batch_val_auprc_mean}')
        print('------------------------------------------------------------')
        epoch_loss.append(batch_loss_mean)
        epoch_val_loss.append(batch_val_loss_mean)
        epoch_val_auroc.append(batch_val_auroc_mean)
        epoch_val_auprc.append(batch_val_auprc_mean)

        #network.save_model()
        
        if (len(epoch_val_loss) > 1) & (epoch_val_loss[-1] == min(epoch_val_loss)):
            if (batch_val_auroc_mean>0.8) & (batch_val_auprc_mean>0.7) :
                # save model
                network.save_model()
        
        if epoch > 1:
            if epoch_val_loss[-1] > epoch_val_loss[-2]:
                patience_value += 1

        if patience_value == patience :
            print('------------------------------------------------')
            print(f'epoch {epoch} End')
            print('================================================')
            patience_value = 0
            break


    
    
    y1 = epoch_val_loss
    y2 = epoch_val_auroc
    y3 = epoch_val_auprc
    x = list(range(1,len(y1)+1))
    
    plt.figure(figsize=(15, 6))

    # 첫 번째 서브플롯
    plt.subplot(1, 3, 1)
    plt.plot(x, y1, marker='x', label='Epoch Val loss', color='limegreen', linestyle='-')
    plt.title("Epoch Val loss")
    plt.xlabel("epoch")
    plt.ylabel("Val_loss")
    plt.legend()

    # 두 번째 서브플롯
    plt.subplot(1, 3, 2)
    plt.plot(x, y2, marker='o', label='Epoch Val AUROC', color='blue', linestyle='-')
    plt.title("Epoch Val AUROC")
    plt.xlabel("epoch")
    plt.ylabel("Val_auroc")
    plt.legend()

    # tp 번째 서브플롯
    plt.subplot(1, 3, 3)
    plt.plot(x, y3, marker='o', label='Epoch Val AUPRC', color='blue', linestyle='-')
    plt.title("Epoch Val AUPRC")
    plt.xlabel("epoch")
    plt.ylabel("Val_auprc")
    plt.legend()

    #plt.tight_layout()  # 서브플롯 간의 간격 조정
    plt.show()

    #validataion loss 추가
    epoch_loss_mean = np.mean(epoch_loss)
    epoch_val_loss_mean = np.mean(epoch_val_loss)
    epoch_val_auroc_mean = np.mean(epoch_val_auroc)
    epoch_val_auprc_mean = np.mean(epoch_val_auprc)
    print('Total val_loss_mean: ', epoch_val_loss_mean)
    print('Total val_auroc_mean: ', epoch_val_auroc_mean)
    print('Total val_auprc_mean: ', epoch_val_auprc_mean)
    #print(f'Total {Total_num} loss : {epoch_loss_mean}')
    #print(f'Total {Total_num} val_loss : {epoch_val_loss_mean}')
    #print('============================================================')

    return epoch_val_auroc_mean

In [ ]:
# Train 데이터셋 생성
train_states, train_actions, train_rewards, train_next_states, train_terminal = make_transition(train_data)

# PyTorch DataLoader로 데이터셋을 불러옵니다.
train_transition = TensorDataset(torch.tensor(train_states, dtype=torch.float32),
                        torch.tensor(train_actions, dtype=torch.long),  # action을 long으로 변환
                        torch.tensor(train_rewards, dtype=torch.float32),
                        torch.tensor(train_next_states, dtype=torch.float32),
                        torch.tensor(train_terminal, dtype=torch.float32))

# val 데이터셋 생성

val_states, val_actions, val_rewards, val_next_states, val_terminal = make_transition(val_data)

# PyTorch DataLoader로 데이터셋을 불러옵니다.
val_transition = TensorDataset(torch.tensor(val_states, dtype=torch.float32),
                        torch.tensor(val_actions, dtype=torch.long),  # action을 long으로 변환
                        torch.tensor(val_rewards, dtype=torch.float32),
                        torch.tensor(val_next_states, dtype=torch.float32),
                        torch.tensor(val_terminal, dtype=torch.float32))

#make_transition_test
batch_size= len(val_data)

s_col = [x for x in val_data.columns if x[:2]=='s:']
a_col = [x for x in val_data.columns if x[:2]=='a:']
r_col = [x for x in val_data.columns if x[:2]=='r:']
label_col = [x for x in val_data.columns if x=='sh:Shock']
t_col = [x for x in val_data.columns if x[:2]=='t:']
dict = {}
dict['traj'] = {}

s = []
a = []
label = []

for traj in tqdm(val_data.stay_id.unique()):
    df_traj = val_data[val_data['stay_id'] == traj]
    dict['traj'][traj] = {'s':[],'a':[],
                        'label':[]}
    dict['traj'][traj]['s'] = df_traj[s_col].values.tolist()
    dict['traj'][traj]['a'] = df_traj[a_col].values.tolist()
    dict['traj'][traj]['label'] = df_traj[label_col].values.tolist()
    
    for step in range(len(df_traj)):
        s.append(dict['traj'][traj]['s'][step])
        a.append(dict['traj'][traj]['a'][step])
        label.append(dict['traj'][traj]['label'][step])

s = torch.FloatTensor(np.float32(s))
a = torch.LongTensor(np.int64(a))
label = torch.LongTensor(np.int64(label))

Dataset = TensorDataset(s,a,label)
                        
total_val_transition = DataLoader(Dataset,batch_size,shuffle=False)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import os
import optuna

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= '0'
os.environ['CUDA_LAUNCH_BLOCKING']= '1'
n_gpu             = 1
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
USE_GPU = torch.cuda.is_available()
print(device)

# Set parameters

study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction="maximize")
study.optimize(objective, n_trials = 3)

pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
import warnings
warnings.filterwarnings('ignore')
batch_size = 256
update_interval = 6
learning_rate = 0.0001
epoch_num = 200

network = C51OffDQNAgent(state_dim=state_dim, action_dim=action_dim, learning_rate=learning_rate, update_interval=update_interval, hidden_sizes=[167, 196, 121, 57], gamma=0.8)
print(network.policy)

result = train_and_evaluate_model(network, train_transition, val_transition, batch_size, epoch_num)

'''
Param 1:
state_dim:  30
action_dim:  6
learning_rate:  0.0001
hidden_sizes:  {'hidden': [167, 196, 121, 57]}
gamma:  0.8
batch_size:  128
epoch_num:  50
update_interval:  6
Utilizing device cuda:1
--------------------------------------------------
'''

In [ ]:
val_states, val_actions, val_rewards, val_next_states, val_terminal = make_transition(val_data)

# PyTorch DataLoader로 데이터셋을 불러옵니다.
val_transition = TensorDataset(torch.tensor(val_states, dtype=torch.float32),
                        torch.tensor(val_actions, dtype=torch.long),  # action을 long으로 변환
                        torch.tensor(val_rewards, dtype=torch.float32),
                        torch.tensor(val_next_states, dtype=torch.float32),
                        torch.tensor(val_terminal, dtype=torch.float32))

#make_transition_test
batch_size= 1

stay_id_col = [x for x in val_data.columns if x=='stay_id']
s_col = [x for x in val_data.columns if x[:2]=='s:']
a_col = [x for x in val_data.columns if x[:2]=='a:']
r_col = [x for x in val_data.columns if x[:2]=='r:']
label_col = [x for x in val_data.columns if x=='sh:Shock']
t_col = [x for x in val_data.columns if x[:2]=='t:']
dict = {}
dict['traj'] = {}

id = []
s = []
a = []
label = []

for traj in tqdm(np.array([30004798])):
    df_traj = val_data[val_data['stay_id'] == traj]
    dict['traj'][traj] = {'id':[],'s':[],'a':[],'label':[]}
    dict['traj'][traj]['id'] = df_traj[stay_id_col].values.tolist()
    dict['traj'][traj]['s'] = df_traj[s_col].values.tolist()
    dict['traj'][traj]['a'] = df_traj[a_col].values.tolist()
    dict['traj'][traj]['label'] = df_traj[label_col].values.tolist()
    
    for step in range(len(df_traj)):
        id.append(dict['traj'][traj]['id'][step])
        s.append(dict['traj'][traj]['s'][step])
        a.append(dict['traj'][traj]['a'][step])
        label.append(dict['traj'][traj]['label'][step])

id = torch.FloatTensor(np.int64(id))
s = torch.FloatTensor(np.float32(s))
a = torch.LongTensor(np.int64(a))
label = torch.LongTensor(np.int64(label))

Dataset = TensorDataset(s,a,label,id)
                        
total_val_transition_for_plot_alive = DataLoader(Dataset,batch_size,shuffle=False)

In [ ]:
# val 데이터셋 생성
val_states, val_actions, val_rewards, val_next_states, val_terminal = make_transition(val_data)

# PyTorch DataLoader로 데이터셋을 불러옵니다.
val_transition = TensorDataset(torch.tensor(val_states, dtype=torch.float32),
                        torch.tensor(val_actions, dtype=torch.long),  # action을 long으로 변환
                        torch.tensor(val_rewards, dtype=torch.float32),
                        torch.tensor(val_next_states, dtype=torch.float32),
                        torch.tensor(val_terminal, dtype=torch.float32))

#make_transition_test
batch_size= 1

stay_id_col = [x for x in val_data.columns if x=='stay_id']
s_col = [x for x in val_data.columns if x[:2]=='s:']
a_col = [x for x in val_data.columns if x[:2]=='a:']
r_col = [x for x in val_data.columns if x[:2]=='r:']
label_col = [x for x in val_data.columns if x=='sh:Shock']
t_col = [x for x in val_data.columns if x[:2]=='t:']
dict = {}
dict['traj'] = {}

id = []
s = []
a = []
label = []

for traj in tqdm(np.array([30025720])):
    df_traj = val_data[val_data['stay_id'] == traj]
    dict['traj'][traj] = {'id':[],'s':[],'a':[],'label':[]}
    dict['traj'][traj]['id'] = df_traj[stay_id_col].values.tolist()
    dict['traj'][traj]['s'] = df_traj[s_col].values.tolist()
    dict['traj'][traj]['a'] = df_traj[a_col].values.tolist()
    dict['traj'][traj]['label'] = df_traj[label_col].values.tolist()
    
    for step in range(len(df_traj)):
        id.append(dict['traj'][traj]['id'][step])
        s.append(dict['traj'][traj]['s'][step])
        a.append(dict['traj'][traj]['a'][step])
        label.append(dict['traj'][traj]['label'][step])

id = torch.FloatTensor(np.int64(id))
s = torch.FloatTensor(np.float32(s))
a = torch.LongTensor(np.int64(a))
label = torch.LongTensor(np.int64(label))

Dataset = TensorDataset(s,a,label,id)
                        
total_val_transition_for_plot_dead = DataLoader(Dataset,batch_size,shuffle=False)

In [ ]:
# val 데이터셋 생성
val_states, val_actions, val_rewards, val_next_states, val_terminal = make_transition(val_data)

# PyTorch DataLoader로 데이터셋을 불러옵니다.
val_transition = TensorDataset(torch.tensor(val_states, dtype=torch.float32),
                        torch.tensor(val_actions, dtype=torch.long),  # action을 long으로 변환
                        torch.tensor(val_rewards, dtype=torch.float32),
                        torch.tensor(val_next_states, dtype=torch.float32),
                        torch.tensor(val_terminal, dtype=torch.float32))

#make_transition_test
batch_size= 1

stay_id_col = [x for x in val_data.columns if x=='stay_id']
s_col = [x for x in val_data.columns if x[:2]=='s:']
a_col = [x for x in val_data.columns if x[:2]=='a:']
r_col = [x for x in val_data.columns if x[:2]=='r:']
label_col = [x for x in val_data.columns if x=='sh:Shock']
t_col = [x for x in val_data.columns if x[:2]=='t:']
dict = {}
dict['traj'] = {}

id = []
s = []
a = []
label = []

for traj in tqdm(val_data.stay_id.unique()):
    df_traj = val_data[val_data['stay_id'] == traj]
    dict['traj'][traj] = {'id':[],'s':[],'a':[],'label':[]}
    dict['traj'][traj]['id'] = df_traj[stay_id_col].values.tolist()
    dict['traj'][traj]['s'] = df_traj[s_col].values.tolist()
    dict['traj'][traj]['a'] = df_traj[a_col].values.tolist()
    dict['traj'][traj]['label'] = df_traj[label_col].values.tolist()
    
    for step in range(len(df_traj)):
        id.append(dict['traj'][traj]['id'][step])
        s.append(dict['traj'][traj]['s'][step])
        a.append(dict['traj'][traj]['a'][step])
        label.append(dict['traj'][traj]['label'][step])

id = torch.FloatTensor(np.int64(id))
s = torch.FloatTensor(np.float32(s))
a = torch.LongTensor(np.int64(a))
label = torch.LongTensor(np.int64(label))

Dataset = TensorDataset(s,a,label,id)
                        
total_val_transition_for_plot = DataLoader(Dataset,batch_size,shuffle=False)

Alive stay_id 찾아서 plot 출력하기

In [ ]:
index_num = 0
val_batch_auroc = []
val_data_df = val_data.copy()
val_data_df['Action by RL max'] = None
val_data_df['Action by RL over mean'] = None
transition = 1
for val_i_batch_auroc, val_batch_data_auroc in enumerate(total_val_transition_for_plot_alive):
    print(f'Transition {transition}')
    val_states, val_actions, val_labels, val_stay_id = val_batch_data_auroc
    action = network.plot_q_value_distribution(state=val_states)
    transition += 1

Dead stay_id 찾아서 plot 출력하기

In [ ]:
index_num = 0
val_batch_auroc = []
val_data_df = val_data.copy()
val_data_df['Action by RL max'] = None
val_data_df['Action by RL over mean'] = None
transition = 1
for val_i_batch_auroc, val_batch_data_auroc in enumerate(total_val_transition_for_plot_dead):
    print(f'Transition {transition}')
    val_states, val_actions, val_labels, val_stay_id = val_batch_data_auroc
    action = network.plot_q_value_distribution(state=val_states)
    transition += 1

stay_id 찾아서 plot 출력하기

In [ ]:
'''index_num = 0
val_batch_auroc = []
val_data_df = val_data.copy()
val_data_df['Action by RL max'] = None
val_data_df['Action by RL over mean'] = None
transition = 1
for val_i_batch_auroc, val_batch_data_auroc in enumerate(total_val_transition_for_plot):
    print(f'Transition {transition}')
    val_states, val_actions, val_labels = val_batch_data_auroc
    action = network.plot_q_value_distribution(state=val_states)
    transition += 1
    break'''

In [ ]:
'''index_num = 0
val_batch_auroc = []
val_data_df = val_data.copy()
val_data_df['Action by RL max'] = None
val_data_df['Action by RL over mean'] = None'''
transition = 1
for val_i_batch_auroc, val_batch_data_auroc in enumerate(total_val_transition_for_plot):
    print(f'Transition {transition}')
    val_states, val_actions, val_labels = val_batch_data_auroc
    q_dist, q_value_mean = network.print_q_value_distribution(state=val_states)

    print('Q value distribution shape: ', q_dist.shape)
    print('Q value distribution type: ', type(q_dist))
    print('Q value distributions: ', q_dist)
    print('-----------------------------------------------------')
    print('Q value mean shape: ', q_value_mean.shape)
    print('Q value mean type: ', type(q_value_mean))
    print('Q value mean: ', q_value_mean)
    print('=====================================================')

    transition += 1
    break